## London Housing Price - Case Study

## Question:  which boroughs of London have seen the greatest increase in housing prices, on average, over the last two decades?

## 1. Sourcing and Loading

### 1.1 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


### 1.2 Download data

In [ ]:
# First, make a variable called url_LondonHousePrices
url_LondonHousePrices = "https://data.london.gov.uk/download/uk-house-price-index/70ac0766-8902-4eb5-aab5-01951aaed773/UK%20House%20price%20index.xls"

# The dataset we're interested in contains the Average prices of the houses, and is actually on a particular sheet of the Excel file. 
# As a result, we need to specify the sheet name in the read_excel() method.
# Put this data into a variable called properties.  
properties = pd.read_excel(url_LondonHousePrices, sheet_name='Average price', index_col= None)

## 2. Cleaning, Transforming, and Visualizing

### 2.1 Exploring Data

In [ ]:
print(properties.head())
# 320 x 49 DataFrame
# the first column contains the date, each subsequent column is a borough 
# the first row is the borough ID and each subsequent row is the borough housing price for that given date
# 320 rows but only 319 are data because the first row is the ID
# 3 empty columns 

In [ ]:
print(properties.info())

### 2.2 Cleaning Data

In [ ]:
test = properties.copy()
col_names = test.pop('Unnamed: 0').copy()
col_names.iloc[0] = 'id'
# transpose the dataframe so that each observations are rows and features are columns
# the borough names are new indices 
test = test.transpose()
test = test.dropna(axis = 0, how = 'any') # important to check the size after this step because it wont raise an error 

# change the column names
# the DF now has size 45 x 320 
test.set_axis(col_names, axis = 1, inplace= True) # set inplace = True to save changes to the DF
print(test.head())
# print(test.shape)

In [ ]:
print(test.index)

In [ ]:
# there are only 32 London boroughs, need to check for duplicates
test.duplicated() # there was no duplicate, so there must be some non-boroughs in this list
# here is a list of london boroughs from Wikipedia, note that city of london is not a london borough
london_boroughs = {1: 'Barking & Dagenham', 2: 'Barnet', 3: 'Bexley', 4: 'Brent', 5: 'Bromley', 
                   6: 'Camden', 7: 'Croydon', 8: 'Ealing', 9: 'Enfield', 10: 'Greenwich', 
                   11: 'Hackney', 12: 'Hammersmith & Fulham', 13: 'Haringey', 14: 'Harrow', 15: 'Havering', 
                   16: 'Hillingdon', 17: 'Hounslow', 18: 'Islington', 19: 'Kensington & Chelsea', 20: 'Kingston upon Thames', 
                   21: 'Lambeth', 22: 'Lewisham', 23: 'Merton', 24: 'Newham', 25: 'Redbridge', 
                   26: 'Richmond upon Thames', 27: 'Southwark', 28: 'Sutton', 29: 'Tower Hamlets',30: 'Waltham Forest',
                   31: 'Wandsworth', 32: 'Westminster'} 

In [ ]:
# subset london boroughs only
test_boroughs = test.loc[london_boroughs.values()] # 32 by 320 DF
# add a borough name
test_boroughs['boroughs'] = london_boroughs.values()
print(test_boroughs.head())

### 2.3 Transforming data

In [ ]:
# tranform the data from wide to long format
test_long = test_boroughs.melt( id_vars = ['boroughs','id'], var_name = 'date', value_name = 'price')

# set price to float 
test_long['price'] = test_long['price'].astype('float')

# extract day, month, and year from TimeStamp objects in pandas
test_long['year'] = [pd.to_datetime(s).year for s in test_long['date']]
test_long['month'] = [pd.to_datetime(s).month for s in test_long['date']]
test_long['day'] = [pd.to_datetime(s).day for s in test_long['date']]

In [ ]:
# get some summary statistics 
print(test_long.pivot_table(values = 'price', index = ['boroughs', 'year'], 
                            aggfunc= [np.mean, np.min, np.max, np.median]))


In [ ]:
#make sure that there was no missing values 
test_long.isna().any()

In [ ]:
properties_clean = test_long

### 2.4 Visualizing Data

In [ ]:
# create a avg_price vs. year plot for each boroughs 
fig, ax = plt.subplots()
price_trend = {}
for key1, row in properties_clean.groupby(['boroughs']):
    year = []
    avg_price = []
    for key2, row2 in row.groupby('year')['price']:
        year.append(key2)
        avg_price.append(row2.mean())
    ax.plot(year, avg_price, label = key1)
    price_trend[key1] = avg_price
plt.legend(bbox_to_anchor=(1.05, 1))
plt.grid()
plt.show()

In [ ]:
# find the boroughs with highest avg_price increase 
price_trend = (pd.DataFrame(price_trend)).transpose()
price_trend.set_axis(properties_clean['year'].unique(), axis = 1, inplace= True)
price_trend['price_diff'] = price_trend[2021] - price_trend[1995]


In [ ]:
print(price_trend.sort_values(2021, ascending = False).iloc[:,[0, -2, -1]])

## 3. Modeling

In [ ]:
# there is no need to write the create_price_ratio because all of the grouping steps have been done
price_trend['18_98_price_ratio'] = price_trend[2018]/price_trend[1998]
print(price_trend.sort_values('18_98_price_ratio', ascending = False).iloc[:,[0, -3, -2, -1]])

In [ ]:
price_trend.plot(kind = "scatter", y = '18_98_price_ratio', x = 'price_diff')
# add annotations
for label, corrs in price_trend.iterrows():
    plt.annotate(label, (corrs['price_diff'], corrs['18_98_price_ratio']))
plt.show()

In [ ]:
price_trend.plot(kind = "scatter", y = '18_98_price_ratio', x = 1995)
# add annotations
for label, corrs in price_trend.iterrows():
    plt.annotate(label, (corrs[1995], corrs['18_98_price_ratio']))
plt.xlabel('1995 price')
plt.title('18_98 price ratio vs housing price in 1995')
plt.show()

## Conclusions

From 1995 to 2021, we saw an average price increase of 456,872 GBP across 32 London boroughs with a highest
absolute increase happened in Kensington & Chelsea of almost 1 million and a the smallest absolute increase happened in Barking & Dagenham around 263k. 

However, when looking at the price ratio between 2018 and 1998, the fastest increase happened in Hackey where housing value increased almost 6 times in 2 decades and the slowest growth was in Houslow of slightly below 4 times. If we segment the data by their values in 1995, the least expensive boroughs - such as Hackney, Waltham Forest, Southwark, underwent the fastest growth in percentage points. However, some equally less expensive boroughs in 1995 such as Houslow, Sutton,  Harrow experienced a much slower growth in 2 decades. The interesting thing is that the growth trends varied so much in the last 2 decades among boroughs with similar values in 1995. 

It is important to remember that changes in housing price is strongly correlated with infrastructure, socialeconomic, education, and commercial developments happening in the corresponding boroughs in the last 2 decades. Many of the least expensive neighborhoods have become a lot more desirable while demands in the expensive boroughs seemed to slow down. 
